In [107]:
%matplotlib inline
import logging
import Euclid
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np

from matplotlib import colors
from matplotlib.gridspec import GridSpec
from astropy.table import Table, Column, join
from tqdm.notebook import tqdm
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks
from scikitplot.metrics import plot_confusion_matrix
from IPython.display import set_matplotlib_formats

%elogin
%erun SIM_Tools

from SimSpectra.SimSpectra import SimGalaxySpectra
from ST_DM_MDBTools.Mdb import Mdb

plt.rcParams['figure.figsize'] = (10, 6)
set_matplotlib_formats('pdf')

In [122]:
pp = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210201.fits', hdu=1)
true = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210129_with_tu_SNR10.fits', hdu=1)
phz = Table.read('/home/aalvarez/Work/Data/SC8/run5/EUC_PHZ_PHZCAT__20210114T191139.733514Z_00.00.fits', hdu=1)
zbins = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210129.fits', hdu=2)['BINS_PDF'].data
mbins = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210129.fits', hdu='BINS_MC_PDF_1D_LOG_STELLARMASS')['BINS_PDF'].data
bins2d = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210201.fits', hdu='BINS_MC_PDF_2D_Z_LOG_STELLARMASS')

In [123]:
joint = join(pp, phz, 'OBJECT_ID')
joint = join(joint, true, 'OBJECT_ID')

In [124]:
idx = np.random.choice(len(joint), 4)

fig, axes = plt.subplots(2, 2)
for ax, obj in zip(axes.ravel(), idx):
    ax.plot(bins, joint['MC_PDF_1D_Z'][obj], label='MC PDZ')
    ax.plot(np.linspace(0, 6, 601), joint['PHZ_PDF'][obj], label='Coadded PDZ')
    ax.axvline(joint['PHZ_PP_MODE_Z'][obj], c='green', linestyle='--', label='MC Z')
    ax.axvline(joint['PHZ_MODE_1'][obj], c='cyan', linestyle='--', label='Coadded Z')
    ax.axvline(joint['Z_OBS'][obj], linestyle=':', color='red', label='True Z')
    ax.set_xlim(0, 2.0)
    ax.legend()

<Figure size 720x432 with 4 Axes>

In [213]:
obj = 100

fig = plt.figure()
gs = GridSpec(ncols=2, nrows=2, hspace=0, wspace=0, height_ratios=[0.75, 0.25], width_ratios=[0.75, 0.25])

grid = np.vstack([bins2d['Z'], bins2d['LOG_STELLARMASS']])
kde = gaussian_kde(grid, weights=joint['MC_PDF_2D_Z_LOG_STELLARMASS'][obj])

de2d = kde(grid).reshape(59, -1)
xx = bins2d['Z'].reshape(59, -1)
yy = bins2d['LOG_STELLARMASS'].reshape(59, -1)

ax2d = fig.add_subplot(gs[0,0])
ax2d.contourf(xx, yy, de2d, cmap='Blues', linestyles=None)
ax2d.set_ylabel('Log stellar mass')
ax2d.axvline(joint['Z_OBS'][obj], linestyle=':', color='red')
ax2d.text(joint['Z_OBS'][obj] + 0.1, 2., 'True Z')
plt.setp(ax2d.get_xticklabels(), visible=False)

axz = fig.add_subplot(gs[1,0], sharex=ax2d)
kde = gaussian_kde(zbins, weights=joint['MC_PDF_1D_Z'][obj])
de = kde(zbins)
axz.plot(zbins, de)
axz.bar(
    zbins, de.max() * joint['MC_PDF_1D_Z'][obj]/joint['MC_PDF_1D_Z'][obj].max(),
    width=np.mean(np.diff(zbins)),
    label='Histogram',
    color='none', edgecolor='orange'
)
axz.set_yticks([])
axz.set_xlabel('Z')
axz.axvline(joint['Z_OBS'][obj], linestyle=':', color='red')

axm = fig.add_subplot(gs[0,1], sharey=ax2d)
kde = gaussian_kde(mbins, weights=joint['MC_PDF_1D_LOG_STELLARMASS'][obj])
de = kde(mbins)
axm.plot(de, mbins, label='Gaussian KDE')
axm.barh(
    mbins, joint['MC_PDF_1D_LOG_STELLARMASS'][obj]/joint['MC_PDF_1D_LOG_STELLARMASS'][obj].max(),
    height=np.mean(np.diff(mbins)),
    label='Histogram',
    color='none', edgecolor='orange'
)
axm.set_xticks([])
plt.setp(axm.get_yticklabels(), visible=False)
axm.legend()

plt.show()

<Figure size 720x432 with 3 Axes>